In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
#import studio.plotly as py
import plotly.tools as tls

import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')
#print(sns.axes_style)

In [ ]:
#df = pd.read_csv("/content/drive/My Drive/Practicum: Identifying meat cut using Ensemble techniques/Dataset.csv")
df = pd.read_csv('/content/drive/My Drive/Practicum: Identifying meat cut using Ensemble techniques/F_names.csv')
#df[['0', '1', '2']] = df[['0', '1', '2']].astype('float64')
df.head()

,Unnamed: 0,FileName,ProductID,density,xmax,xmin,ymax,ymin,xmaxY,xminY,ymaxX,yminX,outFile,Weight
0,0,1500276892.jpg,20001,32196712,489,76,513,57,306,298,346,283,1500276892.jpg,5.94
1,1,1500277002.jpg,20001,38163151,561,67,590,117,396,280,296,241,1500277002.jpg,6.20
2,2,1500277036.jpg,20001,35409978,483,26,445,23,285,267,196,224,1500277036.jpg,6.04
3,3,1500277171.jpg,20001,35608799,524,54,498,81,321,217,331,326,1500277171.jpg,6.48
4,4,1500277182.jpg,20001,38085989,538,78,530,101,267,330,264,384,1500277182.jpg,6.34


In [ ]:
df.count()

Unnamed: 0    8292
FileName      8292
ProductID     8292
density       8292
xmax          8292
xmin          8292
ymax          8292
ymin          8292
xmaxY         8292
xminY         8292
ymaxX         8292
yminX         8292
outFile       8292
Weight        8068
dtype: int64

In [ ]:
df.dropna(subset = ["Weight"], inplace=True)

In [ ]:
df.count()

Unnamed: 0    8068
FileName      8068
ProductID     8068
density       8068
xmax          8068
xmin          8068
ymax          8068
ymin          8068
xmaxY         8068
xminY         8068
ymaxX         8068
yminX         8068
outFile       8068
Weight        8068
dtype: int64

In [ ]:
features = df[["Weight","xmax","xmin","ymax","ymin","xmaxY","xminY","ymaxX","yminX"]]
#features = features.to_numpy() # converts feature set to numpy array
#target = df[['ProductID']] # converts target column to numpy array
#features.shape, len(target)

In [ ]:
corr =features.corr()
corr.style.background_gradient(cmap='coolwarm')

,Weight,xmax,xmin,ymax,ymin,xmaxY,xminY,ymaxX,yminX
Weight,1.000000,0.666895,0.176183,0.821320,0.094880,0.557385,0.500111,0.382171,0.379713
xmax,0.666895,1.000000,0.358043,0.702685,0.621077,0.586362,0.438811,0.603471,0.509410
xmin,0.176183,0.358043,1.000000,0.444408,0.394738,0.402025,0.224742,0.319310,0.249911
ymax,0.821320,0.702685,0.444408,1.000000,0.380368,0.681243,0.647158,0.435624,0.439590
ymin,0.094880,0.621077,0.394738,0.380368,1.000000,0.385531,0.295691,0.373757,0.394708
xmaxY,0.557385,0.586362,0.402025,0.681243,0.385531,1.000000,0.254439,0.550553,0.266558
xminY,0.500111,0.438811,0.224742,0.647158,0.295691,0.254439,1.000000,0.077018,0.370635
ymaxX,0.382171,0.603471,0.319310,0.435624,0.373757,0.550553,0.077018,1.000000,0.236205
yminX,0.379713,0.509410,0.249911,0.439590,0.394708,0.266558,0.370635,0.236205,1.000000


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from statsmodels.api import add_constant
X=df[["xmax","xmaxY","xminY","Weight"]] 
X = add_constant(X)
y=df['ProductID']
vif = [variance_inflation_factor(X.to_numpy(), i) for i in range(X.to_numpy().shape[1])]

print(vif[1:])


[2.121301877236436, 1.6651256874433473, 1.3832180981228566, 2.176752780580983]


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
logreg = LogisticRegression()
y_score = logreg.fit(X, y)

In [ ]:
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X, y)))

Accuracy of logistic regression classifier on train set: 0.69


In [ ]:
y_pred = logreg.predict_proba(X)[:,1]

In [ ]:
print(type(y_pred))

<class 'numpy.ndarray'>


In [ ]:
df["Log_Reg Prediction"] = y_pred.tolist()

In [ ]:
df.head

<bound method NDFrame.head of       Unnamed: 0  Predicted_Class  Colour_Predition  Log_Reg Prediction
0              0     1.960000e-05      2.010000e-09            0.022018
1              1     2.250000e-06      6.720000e-05            0.018175
2              2     7.370000e-08      8.360000e-07            0.023971
3              3     1.124540e-04      2.580000e-09            0.020056
4              4     9.020000e-07      1.540000e-08            0.017171
...          ...              ...               ...                 ...
8063        8063     4.070000e-08      6.230000e-08            0.006688
8064        8064     1.540000e-07      1.481000e-04            0.023385
8065        8065     3.190000e-06      2.130000e-05            0.002962
8066        8066     7.088510e-04      7.120000e-05            0.011960
8067        8067     1.500000e-06      8.440000e-06            0.008205

[8068 rows x 4 columns]>

In [ ]:
#df_new = df.drop('File Name')
from sklearn.model_selection import train_test_split
RSEED = 30
X=df[["FileName","xmax","xmaxY","xminY","Weight"]]
#X_train1, X_test1, y_train, y_test = train_test_split(finalDf.drop('ProductID', axis=1), finalDf['ProductID'], test_size=0.2, random_state=42)
X_train1, X_test1, y_train, y_test = train_test_split(X, df['ProductID'], test_size=0.2, random_state=48)

In [ ]:
X_train = X_train1.drop('FileName', axis=1)
X_test =  X_test1.drop('FileName', axis=1)

#LOGREG

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
logreg = LogisticRegression()
y_score = logreg.fit(X_train, y_train).decision_function(X_test)

In [ ]:
y_pred_train = logreg.predict(X_train)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))

Accuracy of logistic regression classifier on train set: 0.68


In [ ]:
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.69


In [ ]:
y_pred_test

array([20004, 20010, 20003, ..., 20010, 20004, 20010])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print('====Confusion Matrix=====')
print('\n')
print(confusion_matrix(y_test,y_pred_test))
print('\n')
print('====Classification Report====')
print('\n')
print(classification_report(y_test,y_pred_test))

====Confusion Matrix=====


[[  0   0  82   2 122]
 [  0   0   0   0  18]
 [  0   0 266  33 117]
 [  0   0  15 405   0]
 [  0   0 105   3 446]]


====Classification Report====


              precision    recall  f1-score   support

       20001       0.00      0.00      0.00       206
       20002       0.00      0.00      0.00        18
       20003       0.57      0.64      0.60       416
       20004       0.91      0.96      0.94       420
       20010       0.63      0.81      0.71       554

    accuracy                           0.69      1614
   macro avg       0.42      0.48      0.45      1614
weighted avg       0.60      0.69      0.64      1614



In [ ]:
y_pred_test = logreg.predict_proba(X_test)[:,1]
y_pred_train = logreg.predict_proba(X_train)[:,1]

In [ ]:
type(y_pred_train)

numpy.ndarray

In [ ]:
print(y_pred_test)

[0.01746884 0.00300067 0.02299968 ... 0.00709145 0.01267595 0.00760383]


In [ ]:
test_res = []
for i in range(0,len(y_pred_test)):
  for j in range(0,5):
    k = np.amax(y_pred_test)
    test_res.append(k)
print(test_res)


[0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018, 0.4658242835437018,

In [ ]:
np.argmax(y_pred_test,axis=0)

array([272, 953, 177, 953, 177])

In [ ]:
print(y_pred_train[0,4])
print(y_pred_train[5000,4])

0.4017415582084245
0.4055361378959857


In [ ]:
y_pred_train
print(len(y_pred_train))
print(len(y_pred_test))

6454
1614


In [ ]:
fileName_Col = np.concatenate((X_train1['FileName'],X_test1['FileName']), axis=-1)

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
product_IDS = np.concatenate((y_train,y_test), axis=-1)

In [ ]:
pred_array_LogReg = np.concatenate((y_pred_train,y_pred_test), axis=-1)

In [ ]:
final_dataset_pred = pd.DataFrame({'FileName':fileName_Col, 'ProductID':product_IDS, 'LogReg Predictions':pred_array_LogReg})

In [ ]:
print(final_dataset_pred.head())

           FileName  ProductID  LogReg Predictions
0    1500362477.jpg      20003            0.008734
1    1501154486.jpg      20004            0.022887
2    1502368331.jpg      20003            0.008839
3  1500276485_3.jpg      20002            0.006716
4    1501494672.jpg      20003            0.016539


#DESCISION TREE

In [ ]:
import time
start = time.perf_counter()


from sklearn.tree import DecisionTreeClassifier
Classifier = DecisionTreeClassifier(criterion='gini',random_state=500,max_depth=None,min_samples_leaf=20)
history = Classifier.fit(X,y)

elapsed = time.perf_counter() - start
print('Elapsed %.3f seconds.' % elapsed)

Elapsed 0.033 seconds.


In [ ]:
from sklearn import metrics
print ("Decision Tree Train Accuracy:", metrics.accuracy_score(y, Classifier.predict(X)))

Decision Tree Train Accuracy: 0.7958601883986118


In [ ]:
y_pred_dst=Classifier.predict_proba(X)[:,1]

In [ ]:
df["DTC_Prediction"] = y_pred_dst.tolist()

In [ ]:
df.head

<bound method NDFrame.head of       Unnamed: 0  Predicted_Class  ...  Log_Reg Prediction  DTC_Prediction
0              0     1.960000e-05  ...            0.022018             0.0
1              1     2.250000e-06  ...            0.018175             0.0
2              2     7.370000e-08  ...            0.023971             0.0
3              3     1.124540e-04  ...            0.020056             0.0
4              4     9.020000e-07  ...            0.017171             0.0
...          ...              ...  ...                 ...             ...
8063        8063     4.070000e-08  ...            0.006688             0.0
8064        8064     1.540000e-07  ...            0.023385             0.0
8065        8065     3.190000e-06  ...            0.002962             0.0
8066        8066     7.088510e-04  ...            0.011960             0.0
8067        8067     1.500000e-06  ...            0.008205             0.0

[8068 rows x 5 columns]>

In [ ]:
df.to_csv("/content/drive/My Drive/Practicum: Identifying meat cut using Ensemble techniques/MeatCut_Predictions.csv")

In [ ]:
df1= pd.read_csv("/content/drive/My Drive/Practicum: Identifying meat cut using Ensemble techniques/MeatCut_Predictions.csv")